In [2]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
import seaborn as sns

# Add parent directory to path to import project modules
sys.path.insert(0, os.path.abspath('..'))

from dispatcher import SimulationDispatcher, generate_parameter_grid

# Set seaborn theme for consistent styling
sns.set_theme(style="whitegrid", palette="Set2")
plt.rcParams['figure.dpi'] = 100

In [3]:
# Fixed parameters
LENGTH = 200
VMAX = 4
P_RED = 0.1
T_GREEN = 40
STEPS = 100000
REPLICATIONS = 50
P_CHG = 0.05
P_SKID = 0.0

# Variable parameters
injection_rates = np.linspace(0.05, 1.0, 20)
p_break_values = [0.01, 0.1, 0.2, 0.3]

print(f"Number of injection rate values: {len(injection_rates)}")
print(f"Injection rates: {injection_rates}")
print(f"p_skid values: {p_break_values}")
print(f"Number of replications per configuration: {REPLICATIONS}")
print(f"Total simulations: {len(injection_rates) * len(p_break_values) * REPLICATIONS}")

Number of injection rate values: 20
Injection rates: [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7
 0.75 0.8  0.85 0.9  0.95 1.  ]
p_skid values: [0.01, 0.1, 0.2, 0.3]
Number of replications per configuration: 50
Total simulations: 4000


In [4]:
# Generate all parameter combinations
configs = generate_parameter_grid(
    length_values=[LENGTH],
    vmax_values=[VMAX],
    t_green_values=[T_GREEN],
    injection_rate_values=injection_rates.tolist(),
    p_b_values=p_break_values,
    p_chg_values=[P_CHG],
    p_red_values=[P_RED],
    p_skid_values=[P_SKID],
    steps=STEPS,
    metrics_start_step=0,
    replications=REPLICATIONS
)

print(f"Generated {len(configs)} simulation configurations")
print(f"\nFirst configuration:")
print(configs[0])

output_file = "break_test.csv"

Generated 4000 simulation configurations

First configuration:
SimulationConfig(length=200, vmax=4, t_green=40, injection_rate=0.05, p_b=0.01, p_chg=0.05, p_red=0.1, p_skid=0.0, steps=100000, metrics_start_step=0, config_id=0)


In [5]:
# Create dispatcher and run simulation

dispatcher = SimulationDispatcher(
    output_file=output_file,
    use_multiprocessing=True,
    max_workers=None,  # Use all available CPU cores
    verbose=True
)

# Run all simulations
results = dispatcher.run(configs)

print(f"\nSimulations complete! Results saved to {output_file}")

Starting 4000 simulations...
Using multiprocessing
Max workers: 12
Results will be saved to: break_test.csv
--------------------------------------------------------------------------------
[1/4000] Completed config_id=11 | Elapsed: 4.4s | ETA: 17512.7s
[2/4000] Completed config_id=5 | Elapsed: 4.5s | ETA: 8963.5s
[3/4000] Completed config_id=7 | Elapsed: 4.7s | ETA: 6281.7s
[4/4000] Completed config_id=2 | Elapsed: 4.8s | ETA: 4811.0s
[5/4000] Completed config_id=0 | Elapsed: 4.9s | ETA: 3900.2s
[6/4000] Completed config_id=1 | Elapsed: 4.9s | ETA: 3254.5s
[7/4000] Completed config_id=9 | Elapsed: 4.9s | ETA: 2792.0s
[8/4000] Completed config_id=3 | Elapsed: 5.0s | ETA: 2471.2s
[9/4000] Completed config_id=4 | Elapsed: 5.2s | ETA: 2291.9s
[10/4000] Completed config_id=6 | Elapsed: 5.2s | ETA: 2064.1s
[11/4000] Completed config_id=8 | Elapsed: 5.2s | ETA: 1883.3s
[12/4000] Completed config_id=10 | Elapsed: 5.7s | ETA: 1894.3s
[13/4000] Completed config_id=12 | Elapsed: 8.4s | ETA: 2578.

In [6]:
# Load results from CSV
df = pd.read_csv(output_file)

# Display basic info
print(f"Total rows: {len(df)}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

Total rows: 4000

Columns: ['timestamp', 'config_id', 'length', 'vmax', 't_green', 'injection_rate', 'p_b', 'p_chg', 'p_red', 'p_skid', 'steps', 'metrics_start_step', 'n_lateral', 'n_rear_end', 'n_vehicles', 'throughput', 'lateral_to_rear_end_ratio', 'time', 'total_time', 'completed_vehicles', 'avg_travel_time', 'avg_speed']

First few rows:


,timestamp,config_id,length,vmax,t_green,injection_rate,p_b,p_chg,p_red,p_skid,...,n_lateral,n_rear_end,n_vehicles,throughput,lateral_to_rear_end_ratio,time,total_time,completed_vehicles,avg_travel_time,avg_speed
0,2025-11-05T23:50:13.375103,11,200,4,40,0.05,0.01,0.05,0.1,0.0,...,34,0,4972,4971,0,4.179492,4.179798,4903,55.925760,3.611931
1,2025-11-05T23:50:13.483024,5,200,4,40,0.05,0.01,0.05,0.1,0.0,...,48,0,5006,5005,0,4.290667,4.291039,4907,55.673935,3.628269
2,2025-11-05T23:50:13.712991,7,200,4,40,0.05,0.01,0.05,0.1,0.0,...,33,0,4866,4864,0,4.520140,4.520512,4798,55.678408,3.627977
3,2025-11-05T23:50:13.812301,2,200,4,40,0.05,0.01,0.05,0.1,0.0,...,52,0,4853,4853,0,4.631463,4.631783,4747,55.556351,3.635948
4,2025-11-05T23:50:13.880171,0,200,4,40,0.05,0.01,0.05,0.1,0.0,...,46,0,4933,4930,0,4.731531,4.731914,4837,55.653711,3.629587


In [8]:
# Calculate total accidents and accident probability
df['total_accidents'] = df['n_lateral'] + df['n_rear_end']

# P_acc = Number of accidents / (Time steps * Throughput)
# Avoid division by zero
df['p_acc'] = df.apply(
    lambda row: row['total_accidents'] / (row['steps'] * row['throughput']) 
    if row['throughput'] > 0 else 0, 
    axis=1
)

# Display sample calculations
print("Sample calculations:")
print(df[['injection_rate', 'p_b', 'total_accidents', 'steps', 'throughput', 'p_acc']].head(10))

Sample calculations:
   injection_rate   p_b  total_accidents   steps  throughput         p_acc
0            0.05  0.01               34  100000        4971  6.839670e-08
1            0.05  0.01               48  100000        5005  9.590410e-08
2            0.05  0.01               33  100000        4864  6.784539e-08
3            0.05  0.01               52  100000        4853  1.071502e-07
4            0.05  0.01               46  100000        4930  9.330629e-08
5            0.05  0.01               44  100000        4986  8.824709e-08
6            0.05  0.01               45  100000        5011  8.980243e-08
7            0.05  0.01               64  100000        5167  1.238630e-07
8            0.05  0.01               47  100000        4965  9.466264e-08
9            0.05  0.01               29  100000        4987  5.815119e-08


In [10]:
# Group by injection_rate and p_break, then calculate mean and std
aggregated = df.groupby(['injection_rate', 'p_b']).agg({
    'p_acc': ['mean', 'std', 'count'],
    'throughput': 'mean',
    'total_accidents': 'mean'
}).reset_index()

# Flatten column names
aggregated.columns = ['injection_rate', 'p_break', 'p_acc_mean', 'p_acc_std', 'n_replications', 'throughput_mean', 'accidents_mean']

print(f"Aggregated data shape: {aggregated.shape}")
print(f"\nSample aggregated data:")
aggregated.head(10)

Aggregated data shape: (80, 7)

Sample aggregated data:


,injection_rate,p_break,p_acc_mean,p_acc_std,n_replications,throughput_mean,accidents_mean
0,0.05,0.01,8.612669e-08,1.516076e-08,50,4993.80,43.04
1,0.05,0.10,8.760526e-08,1.355761e-08,50,4989.94,43.72
2,0.05,0.20,8.607235e-08,1.431006e-08,50,4981.16,42.88
3,0.05,0.30,8.251788e-08,1.442378e-08,50,4972.78,41.04
4,0.10,0.01,1.715421e-07,1.012284e-08,50,9990.22,171.38
5,0.10,0.10,1.699343e-07,1.224468e-08,50,9973.44,169.50
6,0.10,0.20,1.682766e-07,1.302566e-08,50,9940.50,167.28
7,0.10,0.30,1.647949e-07,1.380005e-08,50,9881.60,162.84
8,0.15,0.01,2.538768e-07,1.248520e-08,50,14973.14,380.16
9,0.15,0.10,2.509632e-07,1.340638e-08,50,14923.78,374.54


In [11]:
# Create the plot
plt.figure(figsize=(10, 6))

# Define markers and colors for each p_break value
markers = ['s', 'o', '^', 'v']
colors = ['black', 'red', 'blue', 'magenta']

# Plot each p_break value as a separate line
for i, p_break in enumerate(p_break_values):
    data = aggregated[aggregated['p_break'] == p_break]
    
    plt.plot(
        data['injection_rate'], 
        data['p_acc_mean'],
        marker=markers[i],
        color=colors[i],
        linestyle='-',
        linewidth=2,
        markersize=8,
        label=f'p_break={p_break}',
        markerfacecolor=colors[i],
        markeredgecolor=colors[i]
    )

# Formatting
plt.xlabel('α', fontsize=14, fontweight='bold')
plt.ylabel('P_acc', fontsize=14, fontweight='bold')
plt.xlim(0, 1.0)
plt.ylim(bottom=0)
plt.legend(loc='best', frameon=True, fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save the figure
plt.savefig('break_plot.png', dpi=300, bbox_inches='tight')
print("Plot saved as 'break_plot.png'")

plt.show()

Plot saved as 'break_plot.png'


/tmp/ipykernel_39927/2121227126.py:38: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [12]:
# Find peak accident probability for each p_break
print("Peak accident probability (P_acc) for each p_break value:\n")
for p_break in p_break_values:
    data = aggregated[aggregated['p_break'] == p_break]
    max_idx = data['p_acc_mean'].idxmax()
    max_row = data.loc[max_idx]
    
    print(f"p_break = {p_break}:")
    print(f"  Maximum P_acc = {max_row['p_acc_mean']:.6f} at α = {max_row['injection_rate']:.3f}")
    print(f"  Throughput = {max_row['throughput_mean']:.1f}")
    print(f"  Accidents = {max_row['accidents_mean']:.2f}")
    print()

# Overall statistics
print("\nOverall statistics:")
print(f"Total simulations run: {len(df)}")
print(f"Total unique configurations: {len(aggregated)}")
print(f"Replications per configuration: {REPLICATIONS}")
print(f"Injection rate range: [{injection_rates.min():.2f}, {injection_rates.max():.2f}]")
print(f"p_break values tested: {p_break_values}")

Peak accident probability (P_acc) for each p_break value:

p_break = 0.01:
  Maximum P_acc = 0.000001 at α = 1.000
  Throughput = 94090.9
  Accidents = 8947.82

p_break = 0.1:
  Maximum P_acc = 0.000001 at α = 1.000
  Throughput = 91905.2
  Accidents = 8500.12

p_break = 0.2:
  Maximum P_acc = 0.000001 at α = 1.000
  Throughput = 88998.6
  Accidents = 7772.54

p_break = 0.3:
  Maximum P_acc = 0.000001 at α = 1.000
  Throughput = 85499.5
  Accidents = 6946.10


Overall statistics:
Total simulations run: 4000
Total unique configurations: 80
Replications per configuration: 50
Injection rate range: [0.05, 1.00]
p_break values tested: [0.01, 0.1, 0.2, 0.3]
